# Basic depiction of a FEP workflow

In [6]:
import BioSimSpace as BSS
import glob

BSS.__version__

'2020.1.0'

## 1. Load ligands and protein into memory.

Loading ligands and protein structures can be done using BSS API.

In [11]:
protein_path = "inputs/protein/ros1"
ligands_path = "inputs/ligands/*.mol2"

# load the protein.
protein = BSS.IO.readPDB(protein_path)
protein

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/tmph8011xru/INPUTS/PROTEIN/ROS1.pdb'

## 2. Parameterise ligands and protein.


## 3. Determine which edges to run using LOMAP.


## 4. Set up SOMD simulation folders.


## 5. Gather MBAR results from simulation outputs.

Here insert a BSS snippet that will somehow take SOMD folders, load them, compute freenrg and MBAR. 

## 6. Compute statistics and plot validations.